<a href="https://www.kaggle.com/code/sivamanivallabhani/bertfinetuning?scriptVersionId=175834563" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Regular imports
import matplotlib.pyplot as plt
import torch
from torch import nn
import numpy as np 
device = "cuda" if torch.cuda.is_available() else "cpu"

# Import torch.nn.functional as F
import torch.nn.functional as F

In [2]:
# Define data path
data_path = "/kaggle/input/olidv1/olid-training-v1.0.tsv"


In [3]:
# Load data
import pandas as pd
data = pd.read_csv(data_path, sep='\t')
df = pd.DataFrame(data)
df.columns = ['id', 'tweet', 'subtask_a', 'subtask_b', 'subtask_c']
df["classk"] = df["subtask_a"].map({"OFF":1, "NOT":0})  # Convert labels to binary (hate vs not hate)
tweets = df.tweet
res = []


In [4]:
df.head()

,id,tweet,subtask_a,subtask_b,subtask_c,classk
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,1
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,1
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,0
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,1
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,0


In [5]:
# Process tweets
for i in range(len(tweets)):
    splitt = tweets[i].split("@USER")
    if len(splitt) == 1:
        res.append(splitt[0])
    else:
        res.append(splitt[-1])

tweets = res

In [6]:

# Import tokenizer and BERT model
from transformers import AutoTokenizer, BertModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [7]:
# Define custom dataset class
class Custom_Dataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        
    def __getitem__(self, index: int) -> (torch.Tensor, int):
        X = self.data[index]
        y = self.labels[index]
        return (X, y)
    
    def __len__(self) -> int:
        return len(self.data)


In [8]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tweets, list(df.classk), test_size=0.2, shuffle=True)

# Define train and test datasets and dataloaders
train_data = Custom_Dataset(data=X_train, labels=y_train)
test_data = Custom_Dataset(data=X_test, labels=y_test)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=3, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [9]:
class Model0(nn.Module):
    def __init__(self, model, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.model = model
        self.fc1 = nn.Linear(768, 256)  # Additional layer for fine-tuning
        self.fc2 = nn.Linear(256, 2)     # Output layer

    def forward(self, X):
        inputs = self.tokenizer(X, return_tensors="pt").to(device)
        outputs = self.model(**inputs)
        outputs = outputs.last_hidden_state[0][0].unsqueeze(1)

        
        # Additional layers for fine-tuning
        outputs = torch.relu(self.fc1(outputs.squeeze()))  # Remove extra dimension
        outputs = self.fc2(outputs)

        return outputs


In [10]:
# Instantiate the model
model0 = Model0(model, tokenizer)
model0.to(device)

# Define the optimizer
optimizer = torch.optim.Adam(params=model0.parameters(), lr=0.00002)

In [11]:
# Define the inter and intra space loss functions
def L_inter(spaces):
    means = [torch.mean(spaces[i]) for i in range(spaces.shape[0])]
    loss = torch.tensor(0.0, requires_grad=True)
    for k in range(len(means)):
        cur = torch.tensor(0.0, requires_grad=True)
        for l in range(len(means)):
            if l == k:
                continue
            cur = cur + (1 / (1 - ((means[k] * means[l]))))
        loss = loss + cur
    return loss

def L_intra(spaces):
    loss = torch.tensor(0.0, requires_grad=True)
    for k in range(len(spaces)):
        loss = loss + 1 / torch.var(spaces[k], dim=0)
    return torch.sum(loss) / loss.shape[0]

In [12]:
# Evaluate the model
correct = 0
total = len(y_test)

true = []
preds = []

with torch.no_grad():
    for i, batch in enumerate(test_loader):
#         if i == 100 :
#             break
        X, y = batch
        y_preds = model0(X)

        # Move y tensor to the same device as y_preds
        y = y.to(y_preds.device)

        y_predict = torch.argmax(y_preds)

        true.extend(y.cpu())  # Move true labels back to CPU
        preds.append(y_predict.cpu())  # Move predictions back to CPU

        cur = torch.eq(y_predict, y)

        correct += torch.sum(cur)

print(f"Test Accuracy: {correct/total}")


Test Accuracy: 0.6461480259895325


In [13]:
# Train the model
model0.train()
try:
    for _ in range(3):
        for i, batch in enumerate(train_loader):
#             if i==100:
#                 break
            if i % 2000 == 0:
                print(i)

            X, Y = batch
            loss = 0

            for X, y in zip(X, Y):
                y_preds = model0(X)
                loss += F.cross_entropy(y_preds.view(1, -1).to(device), torch.tensor([y]).to(device))

            loss /= len(Y)
#             loss += 0.7 * L_inter(torch.cat((hate_space, not_hate_space))) + 0.5 * L_intra(torch.stack([hate_space, not_hate_space]))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        

except KeyboardInterrupt:
    print("Training interrupted by the user.")

0
2000
0
2000
0
2000


In [14]:
# Evaluate the model
correct = 0
total = len(y_test)

true = []
preds = []

with torch.no_grad():
    for i, batch in enumerate(test_loader):
#         if i == 100 :
#             break
        X, y = batch
        y_preds = model0(X)

        # Move y tensor to the same device as y_preds
        y = y.to(y_preds.device)

        y_predict = torch.argmax(y_preds)

        true.extend(y.cpu())  # Move true labels back to CPU
        preds.append(y_predict.cpu())  # Move predictions back to CPU

        cur = torch.eq(y_predict, y)

        correct += torch.sum(cur)

print(f"Test Accuracy: {correct/total}")


Test Accuracy: 0.7707703709602356


In [15]:
# Define hate and not_hate words
hate_words = ['Moist', 'Cunt', 'Panties', 'Fuck', 'Hate', 'Nigger', 'Pussy', 'Ass', 
              'Motherfucker', 'Bitch', 'Damn']
not_hate_words = ['Love', 'Peace', 'Kindness', 'Happiness', 'Respect', 'Friendship',
                  'Appreciation', 'Hope', 'Encouragement', 'Support', 'Caring']

In [16]:
temp = []
for word in hate_words:
    inputs = tokenizer(word, return_tensors="pt").to(device)
    outputs = model(**inputs)
    outputs = torch.mean(outputs.last_hidden_state[0][1:], dim=0)
    temp.append(outputs.tolist())

hate_space = torch.tensor(temp).to(device).requires_grad_()

# Process not_hate words
temp = []
for word in not_hate_words:
    inputs = tokenizer(word, return_tensors="pt").to(device)
    outputs = model(**inputs)
    outputs = torch.mean(outputs.last_hidden_state[0][1:], dim=0)
    temp.append(outputs.tolist())

not_hate_space = torch.tensor(temp).to(device).requires_grad_()

In [17]:
import random
l=[[random.uniform(0,1)] for i in range(11)]
l

[[0.7952997558463241],
 [0.34840362640696565],
 [0.5542534970535218],
 [0.666325641961228],
 [0.2257269089742061],
 [0.5332175447404982],
 [0.6814500244894074],
 [0.09555886929252255],
 [0.3090474355852656],
 [0.3014164437817769],
 [0.9722441498445176]]

In [18]:
# next(iter(test_loader)) 
tensor_shape = (11,1)
hate_space_weights = torch.tensor(l).to(device).requires_grad_()
not_hate_space_weights = torch.tensor(l).to(device).requires_grad_()

print(hate_space_weights.device,not_hate_space_weights.is_leaf)

cuda:0 True


In [19]:
hate_space_weights.shape

torch.Size([11, 1])

In [20]:
hate_space_weights.is_leaf

True

In [21]:
import torch.nn.functional as F

class Model1(nn.Module):

    def __init__(self, model, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.model = model

    def forward(self, X):
        inputs = self.tokenizer(X, return_tensors="pt").to(device)
        outputs = self.model(**inputs)
        outputs = outputs.last_hidden_state[0][1:len(outputs.last_hidden_state[0])].transpose(0, 1)#shape=>(768,no of tokens)
        col_norms = torch.norm(outputs, p=2, dim=0, keepdim=True)
        outputs = outputs / col_norms#makes every word resp unit vector 
        hate_att_scores=torch.matmul(hate_space/torch.norm(hate_space,p=2, dim=0, keepdim=True),outputs)#shape=>(11,v)
        not_hate_att_scores=torch.matmul(not_hate_space/torch.norm(not_hate_space,p=2, dim=0, keepdim=True),outputs)#shape=>(11,v)
        Psk_hate = torch.matmul(hate_att_scores.mean(1), hate_space_weights)#shape=>(1)
        Psk_not_hate = torch.matmul(not_hate_att_scores.mean(1), not_hate_space_weights)#shape=>(1)
        output_tensor = torch.cat((Psk_hate, Psk_not_hate), dim=0).requires_grad_()#
        return output_tensor

In [22]:
model1 = Model1(model, tokenizer)

In [23]:
def L_inter(spaces):
    """this function calculates inter space losses between spaces 
    forces the class spaces of different class to move apart
    """
    means = [torch.mean(spaces[i]) for i in range(spaces.shape[0])]
    
    loss = torch.tensor(0.0, requires_grad=True)
    
    for k in range(len(means)):
        cur = torch.tensor(0.0, requires_grad=True)
        for l in range(len(means)):
            if l == k:
                continue
            cur = cur + (1 / (1 - ((means[k] * means[l]))))
        
        loss = loss + cur
    
    return loss
            
            

def L_intra(spaces):
    """"this function calculates intra space loss 
forces the vector representations inside a class space to have high variability(varience)
"""
    loss=torch.tensor(0.0, requires_grad=True)
    for k in range(len(spaces)):
        loss=loss+1/torch.var(spaces[k], dim=0)
    
    return torch.sum(loss)/loss.shape[0]

In [24]:
print(hate_space,not_hate_space)

tensor([[-0.4958,  0.3879,  0.4584,  ...,  0.1642, -0.2841,  0.3593],
        [-0.5212, -0.7194,  0.2324,  ...,  0.2177, -0.1833,  0.1150],
        [-0.3815,  0.3080,  0.2357,  ..., -0.1345,  0.0544, -0.2487],
        ...,
        [ 0.5201,  0.0686,  0.5038,  ..., -0.3075,  0.1384,  0.1164],
        [ 0.3039, -0.1582,  0.7012,  ..., -0.2858,  0.1117, -0.1752],
        [ 0.4498, -0.0386,  0.3705,  ...,  0.0787, -0.0096, -0.1973]],
       device='cuda:0', requires_grad=True) tensor([[ 0.1825, -0.0585,  0.7568,  ..., -0.0337, -0.2765,  0.3506],
        [-0.1288,  0.0716,  0.5815,  ...,  0.1323,  0.0215, -0.0126],
        [-0.0789,  0.0088,  0.8210,  ...,  0.4057, -0.1703,  0.3102],
        ...,
        [-0.5179, -0.1143,  0.3260,  ..., -0.0702, -0.0579,  0.2858],
        [-0.3774,  0.2145,  0.4802,  ...,  0.0824,  0.2672,  0.1634],
        [ 0.0342,  0.3009,  0.3899,  ...,  0.4879,  0.1242,  0.3558]],
       device='cuda:0', requires_grad=True)


In [25]:
print(hate_space_weights,not_hate_space_weights)

tensor([[0.7953],
        [0.3484],
        [0.5543],
        [0.6663],
        [0.2257],
        [0.5332],
        [0.6815],
        [0.0956],
        [0.3090],
        [0.3014],
        [0.9722]], device='cuda:0', requires_grad=True) tensor([[0.7953],
        [0.3484],
        [0.5543],
        [0.6663],
        [0.2257],
        [0.5332],
        [0.6815],
        [0.0956],
        [0.3090],
        [0.3014],
        [0.9722]], device='cuda:0', requires_grad=True)


In [26]:
from sklearn.metrics import precision_score, recall_score,accuracy_score


model1.to(device)

print(hate_space.is_leaf)
optimizer = torch.optim.Adam(params=[hate_space, not_hate_space, hate_space_weights, not_hate_space_weights], lr=0.0002)
model1.train()


curr=0.1
prev=0
pprev=0
for _ in range(5):
    for i, batch in enumerate(train_loader):
#         if i == 100:
#             break
        if i % 2000 == 0:
            print(i)

        x, Y = batch
        loss = 0

        for X, y in zip(x, Y):
            probs = model1(X)
#             print(y_preds)
            #print(f"printing y_preds={y_preds} shape={y_preds.shape}")

            loss+=F.cross_entropy(probs.view(1, -1).to(device), torch.tensor([y]).to(device))
            """explisit soft max is not used to convert to probs is cross 
            entrophy loss has inbuilt softmax  function adding softmax explisity
            will effect adversly by smoothing again on smoothed values will decrese 
            the weightage for class with high prob """
#             break
        loss /= len(x)
        loss += 1*L_inter(torch.cat((hate_space, not_hate_space))) + 1*L_intra(torch.stack([hate_space, not_hate_space]))
            
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()
#         print(hate_space.grad,hate_space_weights.grad)
    correct = 0
    total = len(y_test)

    true = []
    preds = []

    with torch.inference_mode():
        for i, batch in enumerate(test_loader):
#             if i==100:
#                 break

            X, y = batch
            Psk_hate, Psk_not_hate = model1(X)

            y_predict = torch.argmax(torch.tensor([Psk_hate.mean().item(), Psk_not_hate.mean().item()]))

            true.extend(y)
            preds.append(y_predict)

            cur = torch.eq(y_predict, y)

            correct += torch.sum(cur)
    pprev=prev
    prev=curr
    print(f"Test Accuracy: {correct/total}")
    curr=correct/total
    print(f"Accuracy - {accuracy_score(y_test, preds)}")
    print(f"precison score -{precision_score(y_test, preds, average='macro')}")
    print(f"recall score  -{recall_score(y_test, preds, average='macro')}")
  

True
0
2000
Test Accuracy: 0.7741692066192627
Accuracy - 0.7741691842900302
precison score -0.7629817902747226
recall score  -0.718038564202587
0
2000
Test Accuracy: 0.7779456377029419
Accuracy - 0.7779456193353474
precison score -0.758057280895148
recall score  -0.7381062629187307
0
2000
Test Accuracy: 0.7756797671318054
Accuracy - 0.775679758308157
precison score -0.7547096989618418
recall score  -0.7373987842454642
0
2000
Test Accuracy: 0.7749244570732117
Accuracy - 0.7749244712990937
precison score -0.7531951759678113
recall score  -0.7386159247590446
0
2000
Test Accuracy: 0.7745468020439148
Accuracy - 0.774546827794562
precison score -0.7536682888655205
recall score  -0.7352502036442397


In [27]:
print(hate_space_weights,not_hate_space_weights)

tensor([[ 1.0421],
        [-0.0299],
        [ 0.7459],
        [ 0.4924],
        [ 0.4971],
        [ 0.6244],
        [ 0.4321],
        [-0.2815],
        [ 0.0136],
        [-0.0067],
        [ 0.7240]], device='cuda:0', requires_grad=True) tensor([[ 0.5937],
        [ 0.2126],
        [ 0.3614],
        [ 0.7198],
        [ 0.1251],
        [ 0.3787],
        [ 0.4532],
        [-0.0724],
        [ 0.0758],
        [ 0.2908],
        [ 1.0067]], device='cuda:0', requires_grad=True)


In [28]:
print(hate_space,not_hate_space)

tensor([[-1.0667,  1.8004,  1.4000,  ...,  1.0928, -1.2025,  1.3266],
        [-2.7416, -2.5360, -2.2227,  ...,  2.1598, -2.5661,  2.5077],
        [-1.1108,  2.1112,  0.3770,  ..., -0.9406,  1.1855, -0.2348],
        ...,
        [ 2.9198,  2.5622,  2.4181,  ..., -2.4499,  1.9763,  2.5586],
        [ 2.1140, -1.9239,  1.9092,  ..., -2.4762,  1.5393,  1.4468],
        [ 0.8032, -0.8081,  1.0469,  ...,  1.0889,  1.1505, -0.0229]],
       device='cuda:0', requires_grad=True) tensor([[ 0.8927, -1.1758,  2.1566,  ..., -0.7894, -1.3411,  2.0447],
        [ 1.6053, -1.9400, -1.3985,  ..., -1.5474,  2.0259, -1.3599],
        [ 0.7269, -1.5188,  2.4504,  ...,  2.1880, -1.6549,  2.5753],
        ...,
        [-3.5540, -2.9815, -2.3468,  ..., -2.9078, -3.4363, -1.4807],
        [-2.3275,  2.0511, -1.1135,  ..., -1.5156,  1.6611, -1.6413],
        [ 0.4862,  1.4139, -0.4458,  ...,  1.7397,  0.6358,  1.6055]],
       device='cuda:0', requires_grad=True)


In [29]:
def save_bert_model(model,tokenizer,params):
        output_dir = 'Saved/bert'
#         if(params['train_att']):
#             if(params['lambda_attn']>=1):
#                 params['lambda_attn']=int(params['lambda_attn'])

#             output_dir =  output_dir+str(params['supervised_layer_pos'])+'_'+str(params['num_supervised_heads'])+'_'+str(params['num_classes'])+'_'+str(params['lambda_attn'])+'/'
            
#         else:
#             output_dir=output_dir+'_'+str(params['num_classes'])+'/'
#         print(output_dir)
#         # Create output directory if needed
#         if not os.path.exists(output_dir):
#             os.makedirs(output_dir)

#         print("Saving model to %s" % output_dir)

        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
        model_to_save.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)

In [30]:
save_bert_model(model,tokenizer,model.parameters())

In [31]:
torch.save(hate_space, 'hatespace.pt')
torch.save(hate_space_weights, 'hatespace.pt_weights')
torch.save(not_hate_space, 'not_hatespace.pt')
torch.save(not_hate_space_weights, 'not_hatespace_weights.pt')